In [1]:
!pip install pandas numpy torch vector-quantize-pytorch scikit-learn requests tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 4.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import json
import random
from torch.utils.data import DataLoader
from vector_quantize_pytorch import ResidualVQ
from sklearn.preprocessing import MinMaxScaler
from datasets import load_dataset
from tqdm import tqdm
import requests
import gzip
import shutil
import re
import os

# Data Ingestion

In [3]:
dataset = load_dataset("maharshipandya/spotify-tracks-dataset", split="train")
df = dataset.to_pandas()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

dataset.csv:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114000 [00:00<?, ? examples/s]

In [4]:
df = df.dropna(subset=['track_name', 'artists', 'danceability', 'energy', 'valence'])
df = df.drop_duplicates(subset=['track_id'])

In [5]:
df[['track_name', 'danceability', 'energy', 'track_genre']].head(3)

,track_name,danceability,energy,track_genre
0,Comedy,0.676,0.461,acoustic
1,Ghost - Acoustic,0.420,0.166,acoustic
2,To Begin Again,0.438,0.359,acoustic


# Representation Learning

In [6]:
feature_cols = ['danceability', 'energy', 'key', 'loudness', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Normalize features to 0-1 range for the Neural Network
scaler = MinMaxScaler()
features = scaler.fit_transform(df[feature_cols])
features_tensor = torch.tensor(features, dtype=torch.float32)

# Define the RQ-VAE (The "Tokenizer" for music)
# Compresses 10 floats -> sequence of 3 discrete integers
rq = ResidualVQ(
    dim=len(feature_cols),
    codebook_size=256,     # Vocabulary size per position
    num_quantizers=3,      # Length of ID (e.g., <c1><c2><c3>)
    kmeans_init=True,
    kmeans_iters=10
)
# Train quantization (In practice, you'd save this model)
quantized, indices, loss = rq(features_tensor)

# Create mappings
track_to_sid = {}
sid_to_track = {}
track_id_list = df['track_id'].tolist()
indices_np = indices.numpy()

for tid, idx_seq in zip(track_id_list, indices_np):
    # Format: "<12><45><99>"
    sid_str = "".join([f"<{i}>" for i in idx_seq])
    track_to_sid[tid] = sid_str
    sid_to_track[sid_str] = tid

df['semantic_id'] = df['track_id'].map(track_to_sid)

In [7]:
print(f"Generated Semantic IDs. Example: {df['track_name'].iloc[0]} -> {df['semantic_id'].iloc[0]}")

Generated Semantic IDs. Example: Comedy -> <1><31><173>


# SFT Data Generation

In [8]:
sft_rows = []

templates = [
    "I'm looking for a {genre} song.",
    "Play something {genre}.",
    "Can you recommend a {genre} track with {vibe} energy?",
    "I need music for a {genre} playlist."
]

for _, row in tqdm(df.iterrows(), total=len(df)):
    genre = row['track_genre']
    energy_level = "high" if row['energy'] > 0.7 else "chill" if row['energy'] < 0.4 else "moderate"

    # 1. Create Input Text (Instruction)
    import random
    template = random.choice(templates)
    prompt = template.format(genre=genre.replace("-", " "), vibe=energy_level)

    # 2. Create Target Text (Semantic ID)
    target = row['semantic_id']

    # 3. Format for Training (Standard Alpaca/ShareGPT format)
    sft_rows.append({
        "instruction": prompt,
        "input": "",
        "output": target
    })

# Save
with open("sft_train.jsonl", "w") as f:
    for entry in sft_rows:
        f.write(json.dumps(entry) + "\n")

print(f"✅ Saved {len(sft_rows)} SFT examples.")

100%|██████████| 89740/89740 [00:05<00:00, 16036.35it/s]


✅ Saved 89740 SFT examples.


# Integrate real interaction logs

In [9]:
L_HISTORY_URL = "https://zenodo.org/records/6609677/files/userid_trackid_timestamp.tsv.bz2?download=1"
LOCAL_FILE = "listening_history.tsv.bz2"

In [10]:
!wget -O $LOCAL_FILE "$L_HISTORY_URL"

--2025-12-11 06:51:22--  https://zenodo.org/records/6609677/files/userid_trackid_timestamp.tsv.bz2?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 188.185.43.153, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2211449511 (2.1G) [application/octet-stream]
Saving to: ‘listening_history.tsv.bz2’

listening_history.t 100%[===================>]   2.06G  3.47MB/s    in 8m 46s  

2025-12-11 07:00:08 (4.01 MB/s) - ‘listening_history.tsv.bz2’ saved [2211449511/2211449511]



In [11]:
META_URL = "https://huggingface.co/datasets/nicolaus625/cmi/resolve/main/music4all/id_information.csv"
META_FILE = "id_information.csv"

print(f"⬇️ Downloading Music4All Metadata from {META_URL}...")
try:
    response = requests.get(META_URL, stream=True)
    response.raise_for_status()
    with open(META_FILE, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download successful.")
except Exception as e:
    print(f"Error downloading metadata: {e}")

⬇️ Downloading Music4All Metadata from https://huggingface.co/datasets/nicolaus625/cmi/resolve/main/music4all/id_information.csv...
Download successful.


## Process DPO Pairs

In [12]:
def normalize_text(text):
    """Aggressive normalization for fuzzy-like matching"""
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text) # Remove punctuation
    return text.strip()

# Create a "Bridge Key" -> Semantic ID map
# Key = "artist_name|song_title" (Normalized)
spotify_bridge = {}

# Iterate over your Spotify SFT dataframe
for _, row in tqdm(df.iterrows(), total=len(df), desc="Indexing SFT Data"):
    if row['track_id'] in track_to_sid: # Ensure we have a semantic ID for this track
        # Create a normalized lookup key
        key = f"{normalize_text(row['artists'])}|{normalize_text(row['track_name'])}"
        spotify_bridge[key] = track_to_sid[row['track_id']]

print(f"Bridge Index Built. {len(spotify_bridge)} targets available.")

Indexing SFT Data: 100%|██████████| 89740/89740 [00:05<00:00, 15598.35it/s]

Bridge Index Built. 81177 targets available.


In [13]:
df_m4a_meta = pd.read_csv(META_FILE, sep='\t', on_bad_lines='skip', dtype=str)

# Create a map: Music4All_ID -> Bridge_Key
m4a_to_bridge_key = {}
for _, row in tqdm(df_m4a_meta.iterrows(), total=len(df_m4a_meta), desc="Indexing Log Metadata"):
    try:
        # Columns in this file are: id, artist, song, album_name
        if pd.notna(row['artist']) and pd.notna(row['song']):
            key = f"{normalize_text(row['artist'])}|{normalize_text(row['song'])}"
            m4a_to_bridge_key[row['id']] = key
    except:
        continue

print(f"Metadata loaded. Mapped {len(m4a_to_bridge_key)} Music4All IDs to text keys.")

Indexing Log Metadata: 100%|██████████| 109269/109269 [00:08<00:00, 13629.78it/s]

Metadata loaded. Mapped 109269 Music4All IDs to text keys.


In [14]:
# Output file for Tinker
dpo_output_file = "dpo_train.jsonl"
dpo_pairs = []

# Logic Configuration
SKIP_THRESHOLD = 30   # Seconds (Rejected)
ENGAGE_THRESHOLD = 90 # Seconds (Chosen)
CHUNK_SIZE = 500000   # Process in chunks to save RAM

# Ensure the log file is present
LOGS_FILE = "listening_history.tsv.bz2"
if not os.path.exists(LOGS_FILE):
    print(f"⚠️ Warning: {LOGS_FILE} not found. Please re-run the download step.")

print("🔹 Processing Interaction Logs...")

# Stats for report
stats = {"processed": 0, "matches": 0, "positives": 0, "negatives": 0}

# Define a helper to get SID from Music4All ID
def get_sid(m4a_id):
    key = m4a_to_bridge_key.get(m4a_id)
    if key:
        return spotify_bridge.get(key) # Returns Semantic ID or None
    return None

# --- FIX IS HERE: header=0 to skip the first row, names to rename columns ---
with pd.read_csv(
    LOGS_FILE,
    sep='\t',
    header=0,                                  # Tell pandas the first row is a header
    names=['user', 'song', 'timestamp'],       # Rename columns for consistency
    chunksize=CHUNK_SIZE,
    compression='bz2',
    dtype={'user': str, 'song': str},          # Force IDs to string to avoid DtypeWarning
    on_bad_lines='skip'                        # Skip corrupted lines
) as reader:

    for chunk in reader:
        # 1. Preprocessing (with error handling)
        # errors='coerce' turns "timestamp" text or garbage into NaT (Not a Time) so we can drop it
        chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
        chunk = chunk.dropna(subset=['timestamp']) # Drop rows where date failed

        chunk = chunk.sort_values(['user', 'timestamp'])

        # 2. Calculate Dwell Time (Next Timestamp - Current Timestamp)
        chunk['next_ts'] = chunk.groupby('user')['timestamp'].shift(-1)
        chunk['dwell'] = (chunk['next_ts'] - chunk['timestamp']).dt.total_seconds()

        # 3. Map IDs to Semantic IDs (The Bridge)
        chunk['semantic_id'] = chunk['song'].map(get_sid)

        # Drop rows where we couldn't match the song OR calculate dwell time
        matched_chunk = chunk.dropna(subset=['semantic_id', 'dwell'])
        stats['matches'] += len(matched_chunk)

        # 4. Generate Pairs (In-Batch Strategy)
        positives = matched_chunk[matched_chunk['dwell'] > ENGAGE_THRESHOLD]
        negatives = matched_chunk[matched_chunk['dwell'] < SKIP_THRESHOLD]

        stats['positives'] += len(positives)
        stats['negatives'] += len(negatives)

        # Pair Strategy: Match positive with random negative
        if not negatives.empty and not positives.empty:
            negative_pool = negatives['semantic_id'].tolist()

            for _, row in positives.iterrows():
                key = m4a_to_bridge_key.get(row['song'])
                if not key: continue

                # Prompt: "User recently listened to [Artist]..."
                artist_name = " ".join([part.title() for part in key.split("|")]) # Fix: Apply .title() to each part of the split string
                prompt = f"I want to listen to {artist_name} or something similar."

                # Pick a random negative
                rejected_sid = np.random.choice(negative_pool)

                if row['semantic_id'] == rejected_sid: continue

                dpo_pairs.append({
                    "instruction": prompt,
                    "input": "",
                    "chosen": row['semantic_id'],
                    "rejected": rejected_sid
                })

        stats['processed'] += len(chunk)
        print(f"   Processed {stats['processed']:,} logs... DPO Pairs found: {len(dpo_pairs):,}")

        # LIMIT: Stop after gathering 15k pairs to save time
        if len(dpo_pairs) > 15000:
            break

# Save to JSONL
with open(dpo_output_file, "w") as f:
    for entry in dpo_pairs:
        f.write(json.dumps(entry) + "\n")

print(f"\n✅ DONE. Saved {len(dpo_pairs)} DPO pairs to {dpo_output_file}")
print(f"Stats: {stats}")

🔹 Processing Interaction Logs...
   Processed 500,000 logs... DPO Pairs found: 89,700

✅ DONE. Saved 89700 DPO pairs to dpo_train.jsonl
Stats: {'processed': 500000, 'matches': 90537, 'positives': 89796, 'negatives': 502}


# SFT

In [ ]:
!pip install tinker

In [28]:
import os
import json
import asyncio
import numpy as np
import tinker
from tinker import types

# --- Configuration ---
API_KEY = "tml-rlTGuyznut8pATEC9NjKLQhNPe0YeCNR9XYdq93iU31DgScLNuNuE7eTBtsF37xbDAAAA" # Replace with your key
BASE_MODEL = "meta-llama/Llama-3.1-8B"

# Hyperparameters
SFT_LR = 2e-4
DPO_LR = 5e-7
DPO_BETA = 0.1
SFT_EPOCHS = 1
DPO_EPOCHS = 1

# --- Helper: Data Encoding ---
class DataProcessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encode_sft_datum(self, prompt: str, response: str) -> types.Datum:
        """
        Creates a Datum for SFT.
        Masks the prompt so loss is only calculated on the response.
        """
        # Tokenize
        p_ids = self.tokenizer.encode(prompt)
        r_ids = self.tokenizer.encode(response) + [self.tokenizer.eot_token_id]
        input_ids = p_ids + r_ids

        # Create Weight Mask (0.0 for prompt, 1.0 for response)
        # Tinker uses 'weights' in loss_fn_inputs to mask tokens
        weights = [0.0] * len(p_ids) + [1.0] * len(r_ids)

        return types.Datum(
            model_input=types.ModelInput.from_ints(input_ids),
            loss_fn_inputs={
                "target_tokens": types.ModelInput.from_ints(input_ids),
                "weights": types.TensorData.from_numpy(np.array(weights, dtype=np.float32))
            }
        )

    def encode_for_inference(self, text: str) -> types.Datum:
        """Creates a simple datum for forward-pass logprob calculation."""
        ids = self.tokenizer.encode(text)
        return types.Datum(
            model_input=types.ModelInput.from_ints(ids),
            loss_fn_inputs={
                "target_tokens": types.ModelInput.from_ints(ids)
            }
        )

    def encode_dpo_datum(self, prompt: str, chosen: str, rejected: str, ref_logps: dict) -> types.Datum:
        """
        Creates a Datum for DPO.
        Requires pre-computed reference logprobs for stability and efficiency.
        """
        p_ids = self.tokenizer.encode(prompt)
        c_ids = self.tokenizer.encode(chosen) + [self.tokenizer.eot_token_id]
        r_ids = self.tokenizer.encode(rejected) + [self.tokenizer.eot_token_id]

        # Tinker DPO primitive typically expects concatenated sequences or specific keys
        # We pass the tokenized sequences and the pre-computed reference scores
        return types.Datum(
            model_input=types.ModelInput.from_ints(p_ids), # Helper, actual input handled by internal DPO logic or specific keys below
            loss_fn_inputs={
                "chosen_input_ids": types.ModelInput.from_ints(p_ids + c_ids),
                "rejected_input_ids": types.ModelInput.from_ints(p_ids + r_ids),
                # Pass pre-computed reference log sums to avoid re-running ref model during training
                "ref_logps_chosen": types.TensorData.from_numpy(np.array(ref_logps['chosen'], dtype=np.float32)),
                "ref_logps_rejected": types.TensorData.from_numpy(np.array(ref_logps['rejected'], dtype=np.float32)),
                "beta": types.TensorData.from_numpy(np.array(DPO_BETA, dtype=np.float32))
            }
        )

# --- Phase 1: Supervised Fine-Tuning (SFT) ---
async def run_sft(service_client, train_file):
    print(f"Starting SFT on {BASE_MODEL}...")

    # 1. Initialize SFT Client
    sft_client = service_client.create_lora_training_client(
        base_model=BASE_MODEL,
        rank=32
    )
    processor = DataProcessor(sft_client.get_tokenizer())

    # 2. Load and Format Data
    with open(train_file, 'r') as f:
        raw_data = [json.loads(line) for line in f]

    # 3. Training Loop
    for epoch in range(SFT_EPOCHS):
        print(f"  Epoch {epoch+1}/{SFT_EPOCHS}")
        futures = []

        for i, row in enumerate(raw_data):
            datum = processor.encode_sft_datum(row['instruction'], row['output'])

            # Async Forward/Backward
            fut = sft_client.forward_backward(
                data=[datum],
                loss_fn="cross_entropy"
            )
            futures.append(fut)

            # Optimizer Step
            sft_client.optim_step(learning_rate=SFT_LR)

            # Simple logging
            if i % 10 == 0:
                res = await fut # Sync periodically to check status/metrics
                print(f"    Step {i}: Loss {res.metrics['loss']:.4f}")

    # 4. Save SFT Weights
    sft_ckpt_id = sft_client.save_weights(name="sft_final")
    print(f"SFT Completed. Checkpoint: {sft_ckpt_id}")
    return sft_ckpt_id

# --- Phase 2: DPO Pre-computation & Training ---
async def run_dpo(service_client, sft_ckpt_id, dpo_file):
    print(f"Starting DPO using Reference: {sft_ckpt_id}...")

    # 1. Initialize Reference Client (Frozen SFT Model) for Logprobs
    ref_client = service_client.create_lora_training_client(base_model=BASE_MODEL, rank=32)
    await ref_client.load_weights(sft_ckpt_id)
    processor = DataProcessor(ref_client.get_tokenizer())

    # 2. Load Data
    with open(dpo_file, 'r') as f:
        dpo_data = [json.loads(line) for line in f]

    # 3. Pre-compute Reference Logprobs
    # We run a forward pass on the "chosen" and "rejected" responses using the SFT model
    print("  Pre-computing reference logprobs...")
    enriched_data = []
    for row in dpo_data:
        prompt = row['instruction']

        # Encode full sequences
        d_chosen = processor.encode_for_inference(prompt + row['chosen'])
        d_rejected = processor.encode_for_inference(prompt + row['rejected'])

        # Forward pass only (loss_fn='cross_entropy' returns metrics/logprobs without backward if requested/configured)
        # Using specific 'causal_lm_logprobs' string if supported, or extracting from CE outputs
        f_c = ref_client.forward(data=[d_chosen], loss_fn="cross_entropy")
        f_r = ref_client.forward(data=[d_rejected], loss_fn="cross_entropy")

        res_c, res_r = await asyncio.gather(f_c, f_r)

        # Store log sums (assuming API returns 'loss' or explicit 'logprobs')
        # Here we approximate ref_logp using negative loss * length if explicit logsum isn't in metrics
        # Ideally check res_c.outputs['logprobs_sum']
        row['ref_logps'] = {
            'chosen': -res_c.metrics['loss'] * len(d_chosen.loss_fn_inputs['target_tokens'].token_ids),
            'rejected': -res_r.metrics['loss'] * len(d_rejected.loss_fn_inputs['target_tokens'].token_ids)
        }
        enriched_data.append(row)

    # 4. Initialize Policy Client (Trainable)
    # Start from SFT weights
    policy_client = service_client.create_lora_training_client(base_model=BASE_MODEL, rank=32)
    await policy_client.load_weights(sft_ckpt_id)

    # 5. DPO Training Loop
    print("  Starting DPO Loop...")
    for epoch in range(DPO_EPOCHS):
        for i, row in enumerate(enriched_data):
            datum = processor.encode_dpo_datum(
                row['instruction'],
                row['chosen'],
                row['rejected'],
                row['ref_logps']
            )

            # Send DPO Request
            fut = policy_client.forward_backward(
                data=[datum],
                loss_fn="dpo"
            )

            # Step Optimizer (Lower LR)
            policy_client.optim_step(learning_rate=DPO_LR)

            if i % 5 == 0:
                res = await fut
                print(f"    DPO Step {i}: Loss {res.metrics['loss']:.4f}")

    final_id = policy_client.save_weights(name="dpo_aligned_final")
    print(f"DPO Completed. Final Model: {final_id}")
    return final_id

# --- Main Execution Block ---
async def main():
    # Setup
    client = tinker.ServiceClient(api_key=API_KEY)

    # Run SFT
    sft_id = await run_sft(client, "sft_train.jsonl")

    # Run DPO (using SFT result as reference)
    await run_dpo(client, sft_id, "dpo_train.jsonl")

if __name__ == "__main__":
    # Execute the async pipeline
      await main()

Starting SFT on meta-llama/Llama-3.1-8B...
  Epoch 1/1


AttributeError: PreTrainedTokenizerFast has no attribute eot_token_id

## chatgpt

In [ ]:
# Single runnable cell: SFT -> DPO using Tinker cookbook style (adapted to your jsonl formats)
# Run in your notebook. Adjust hyperparams below as desired.

# pip installs (uncomment if necessary)
# !pip install --upgrade tinker tinker_cookbook --quiet

import os
import json
import math
import numpy as np
from tqdm import tqdm
import logging
logging.basicConfig(level=logging.INFO)

# ---------------------------
# User config (edit as needed)
# ---------------------------
os.environ["TINKER_API_KEY"] = "tml-rlTGuyznut8pATEC9NjKLQhNPe0YeCNR9XYdq93iU31DgScLNuNuE7eTBtsF37xbDAAAA"
MODEL_NAME = "meta-llama/Llama-3.2-1B"   # base model
LORA_RANK = 32
LEARNING_RATE = 1e-4
BATCH_SIZE = 4
MAX_LENGTH = 4096
NUM_EPOCHS_SFT = 2
NUM_EPOCHS_DPO = 1
SAVE_NAME_SFT = "song-recommender-SFT"
SAVE_NAME_FINAL = "song-recommender-SFT-DPO"

# ---------------------------
# Imports (cookbook helpers)
# ---------------------------
import tinker
from tinker import types, AdamParams
from tinker_cookbook import renderers, model_info
from tinker_cookbook.tokenizer_utils import get_tokenizer
from tinker_cookbook.supervised.data import conversation_to_datum
from tinker_cookbook.supervised.common import compute_mean_nll

# ---------------------------
# Sanity checks
# ---------------------------
print("Tinker version:", getattr(tinker, "__version__", "unknown"))
service_client = tinker.ServiceClient()

# ---------------------------
# Tokenizer & renderer
# ---------------------------
tokenizer = get_tokenizer(MODEL_NAME)
renderer_name = model_info.get_recommended_renderer_name(MODEL_NAME)
renderer = renderers.get_renderer(renderer_name, tokenizer)

# ---------------------------
# Load SFT JSONL
# Format expected: {"instruction": "...", "input": "", "output": "<1><31><173>"}
# ---------------------------
sft_raw = []
with open("sft_train.jsonl", "r", encoding="utf-8") as f:
    for ln in f:
        if not ln.strip():
            continue
        sft_raw.append(json.loads(ln))
print("Loaded SFT examples:", len(sft_raw))
if len(sft_raw) == 0:
    raise RuntimeError("No SFT examples loaded from sft_train.jsonl")

# Convert SFT examples to Tinker Datum objects using conversation_to_datum
sft_data = []
for ex in tqdm(sft_raw, desc="Converting SFT examples"):
    # Build messages structure expected by conversation_to_datum
    user_text = ex.get("instruction", "")
    input_text = ex.get("input", "")
    if input_text:
        user_text = user_text + "\n" + input_text

    assistant_text = ex.get("output", "")
    messages = [
        {"role": "user", "content": user_text},
        {"role": "assistant", "content": assistant_text}
    ]

    # conversation_to_datum will render & create model_input + loss inputs
    datum = conversation_to_datum(messages, renderer, MAX_LENGTH, renderers.TrainOnWhat.ALL_ASSISTANT_MESSAGES)
    sft_data.append(datum)
print("Converted SFT examples -> datums:", len(sft_data))

# ---------------------------
# Create LoRA training client for SFT
# ---------------------------
print("Creating SFT LoRA training client...")
training_client = service_client.create_lora_training_client(base_model=MODEL_NAME, rank=LORA_RANK)
print("Training client created.")

# ---------------------------
# SFT training loop
# ---------------------------
print("\nStarting SFT training...")
n_batches = max(1, len(sft_data) // BATCH_SIZE)
total_steps = n_batches * NUM_EPOCHS_SFT
training_losses = []

for epoch in range(NUM_EPOCHS_SFT):
    print(f"\n=== SFT Epoch {epoch+1}/{NUM_EPOCHS_SFT} ===")
    np.random.shuffle(sft_data)
    epoch_losses = []

    for b in tqdm(range(n_batches), desc=f"SFT epoch {epoch+1}"):
        batch = sft_data[b * BATCH_SIZE : (b+1) * BATCH_SIZE]

        # linear lr schedule (simple)
        step = epoch * n_batches + b
        lr_mult = max(0.1, 1.0 - step / float(max(1, total_steps)))
        current_lr = LEARNING_RATE * lr_mult

        adam = AdamParams(learning_rate=current_lr, beta1=0.9, beta2=0.95, eps=1e-8)

        # forward_backward and optim_step as in cookbook
        fwd_future = training_client.forward_backward(batch, loss_fn="cross_entropy")
        opt_future = training_client.optim_step(adam)

        fwd_res = fwd_future.result()
        _opt = opt_future.result()

        # compute mean NLL using cookbook helper
        # fwd_res.loss_fn_outputs and batch[i].loss_fn_inputs exist per cookbook
        logprobs = [out["logprobs"] for out in fwd_res.loss_fn_outputs]
        weights = [d.loss_fn_inputs.get("weights", None) for d in batch]
        # compute_mean_nll expects list-of-lists or similar; handle missing weights
        try:
            nll = compute_mean_nll(logprobs, weights)
        except Exception:
            # fallback: average negative logprob if structure differs
            # try to find 'nll' or compute simple average
            flat_vals = []
            for out in fwd_res.loss_fn_outputs:
                # try common keys
                if "nll" in out:
                    flat_vals.append(float(out["nll"]))
                elif "loss" in out:
                    flat_vals.append(float(out["loss"]))
                else:
                    # try sum of token logprobs
                    lp = out.get("logprobs")
                    if lp is None:
                        lp = out.get("token_logprobs") or out.get("prompt_logprobs")
                    if lp:
                        # take negative mean
                        try:
                            flat_vals.append(-float(np.mean(lp)))
                        except Exception:
                            flat_vals.append(0.0)
            nll = float(np.mean(flat_vals)) if flat_vals else 0.0

        epoch_losses.append(nll)
        training_losses.append(nll)

        if b % 20 == 0:
            print(f"  step={b}/{n_batches} | nll={nll:.4f} | lr={current_lr:.6f}")

    print(f"Epoch {epoch+1} avg nll: {np.mean(epoch_losses):.4f}")

print("✅ SFT finished.")

# Save SFT weights and create sampling client for verification and DPO base
print("Saving SFT weights and creating sampling client...")
if hasattr(training_client, "save_weights_and_get_sampling_client"):
    sft_sampling_client = training_client.save_weights_and_get_sampling_client(name=SAVE_NAME_SFT if (SAVE_NAME_SFT := SAVE_NAME_SFT if 'SAVE_NAME_SFT' in globals() else SAVE_NAME_SFT) else "song-recommender-sft")
    print("SFT sampling client created.")
else:
    # fallback: save_state then attempt to get sampling client
    if hasattr(training_client, "save_state"):
        ckpt = training_client.save_state()
        print("Saved state:", ckpt)
    sft_sampling_client = None
    print("No direct save_weights_and_get_sampling_client available; continue to DPO client creation using base model name.")

# ---------------------------
# Load DPO JSONL
# Format expected: {"instruction":"...", "input":"", "chosen":"<...>", "rejected":"<...>"}
# ---------------------------
dpo_raw = []
with open("dpo_train.jsonl", "r", encoding="utf-8") as f:
    for ln in f:
        if not ln.strip():
            continue
        dpo_raw.append(json.loads(ln))
print("Loaded DPO examples:", len(dpo_raw))
if len(dpo_raw) == 0:
    raise RuntimeError("No DPO examples loaded from dpo_train.jsonl")

# ---------------------------
# Helper: build DPO Datum objects programmatically
# ---------------------------
def build_dpo_datum_from_example(example):
    prompt = example.get("instruction", "")
    input_text = example.get("input", "")
    if input_text:
        prompt_text = prompt + "\n" + input_text
    else:
        prompt_text = prompt

    # Build messages for renderer consistency (same as SFT)
    messages = [{"role":"user","content":prompt_text},
                {"role":"assistant","content": ""}]  # assistant content left blank for model_input; completions attached as loss inputs

    # Use renderer to format prompt portion exactly as SFT
    rendered = renderer.render({"messages": messages})
    # tokenize prompt (with special tokens)
    prompt_ids = tokenizer.encode(rendered, add_special_tokens=True)
    # truncate if needed
    if len(prompt_ids) > MAX_LENGTH // 2:
        prompt_ids = prompt_ids[-(MAX_LENGTH//2):]

    # Tokenize chosen/rejected completions (these are your token-id lists like "<146><191><154>")
    # The dataset seems to already encode titles as token-index strings like "<146><191><154>"
    # If those are literal tokens you want to pass through, we treat them as text and encode.
    chosen_text = example.get("chosen", "")
    rejected_text = example.get("rejected", "")

    chosen_ids = tokenizer.encode(" " + chosen_text, add_special_tokens=False)
    rejected_ids = tokenizer.encode(" " + rejected_text, add_special_tokens=False)

    model_input = types.ModelInput.from_ints(prompt_ids)

    loss_inputs = {
        "chosen_tokens": np.array(chosen_ids, dtype=np.int64),
        "rejected_tokens": np.array(rejected_ids, dtype=np.int64),
    }

    datum = types.Datum(
        model_input=model_input,
        loss_fn_inputs=types.Datum.convert_tensors(loss_inputs)
    )
    return datum

# Convert all DPO examples into datums
dpo_datums = [build_dpo_datum_from_example(x) for x in tqdm(dpo_raw, desc="Converting DPO examples")]

# ---------------------------
# Create DPO training client based on SFT checkpoint (if supported)
# Option A: create new LoRA client that uses SFT weights by name "song-recommender-SFT"
# Option B: create client from base model (MODEL_NAME) and then load SFT state - handled by SDK versions differently.
# We'll attempt Option A first (most cookbook examples do this).
# ---------------------------
print("Creating DPO LoRA training client (starting from SFT weights if available)...")
try:
    dpo_client = service_client.create_lora_training_client(base_model=SAVE_NAME_SFT if 'SAVE_NAME_SFT' in globals() else SAVE_NAME_SFT, rank=LORA_RANK)
    print("Created DPO client from SFT name.")
except Exception as e:
    print("Could not create DPO client from SFT name. Falling back to base model. Error:", e)
    dpo_client = service_client.create_lora_training_client(base_model=MODEL_NAME, rank=LORA_RANK)
    # If API allows loading SFT weights explicitly, you can call that here; else DPO will start from base

# ---------------------------
# DPO training loop
# ---------------------------
print("\nStarting DPO training...")
n_batches = max(1, len(dpo_datums) // BATCH_SIZE)
total_steps = n_batches * NUM_EPOCHS_DPO

for epoch in range(NUM_EPOCHS_DPO):
    print(f"\n=== DPO Epoch {epoch+1}/{NUM_EPOCHS_DPO} ===")
    np.random.shuffle(dpo_datums)
    epoch_losses = []

    for b in tqdm(range(n_batches), desc=f"DPO epoch {epoch+1}"):
        batch = dpo_datums[b * BATCH_SIZE : (b+1) * BATCH_SIZE]

        adam = AdamParams(learning_rate=LEARNING_RATE, beta1=0.9, beta2=0.95, eps=1e-8)

        # forward_backward with 'dpo' loss (per cookbook/docs)
        fwd_future = dpo_client.forward_backward(batch, loss_fn="dpo", loss_fn_config={"beta": 0.1})
        opt_future = dpo_client.optim_step(adam)

        fwd_res = fwd_future.result()
        _ = opt_future.result()

        # Try to extract chosen/rejected logprobs or dpo loss from loss_fn_outputs
        outs = fwd_res.loss_fn_outputs
        # outs is a list with one dict per datum; try common keys
        per_example_losses = []
        for out in outs:
            # try to find straightforward loss entry
            if "dpo_loss" in out:
                per_example_losses.append(float(out["dpo_loss"]))
            elif "loss" in out:
                per_example_losses.append(float(out["loss"]))
            else:
                # attempt to compute -log(sigmoid(beta*(lp_chosen - lp_rejected)))
                lp_chosen = None
                lp_rejected = None
                # check several plausible keys
                for k in ("chosen_logprobs", "chosen_lp", "chosen_logprob", "chosen_token_logprobs", "chosen_token_logprob"):
                    if k in out:
                        val = out[k]
                        # val could be list/array -> sum/mean
                        try:
                            lp_chosen = float(np.sum(val)) if hasattr(val, "__len__") else float(val)
                            break
                        except Exception:
                            lp_chosen = None
                for k in ("rejected_logprobs", "rejected_lp", "rejected_logprob", "rejected_token_logprobs", "rejected_token_logprob"):
                    if k in out:
                        val = out[k]
                        try:
                            lp_rejected = float(np.sum(val)) if hasattr(val, "__len__") else float(val)
                            break
                        except Exception:
                            lp_rejected = None

                if lp_chosen is not None and lp_rejected is not None:
                    beta = 0.1
                    z = beta * (lp_chosen - lp_rejected)
                    # numerical stable sigmoid
                    dpo_loss = -math.log(1.0 / (1.0 + math.exp(-z)) + 1e-12)
                    per_example_losses.append(dpo_loss)
                else:
                    # fallback: look for 'nll' or 'logprob' keys
                    if "nll" in out:
                        per_example_losses.append(float(out["nll"]))
                    else:
                        per_example_losses.append(0.0)  # unknown; don't fail

        mean_loss = float(np.mean(per_example_losses)) if per_example_losses else 0.0
        epoch_losses.append(mean_loss)

        if b % 20 == 0:
            print(f"  step={b}/{n_batches} | approx_dpo_loss={mean_loss:.4f}")

    print(f"DPO epoch {epoch+1} avg loss: {np.mean(epoch_losses):.4f}")

print("✅ DPO training complete.")

# ---------------------------
# Save final DPO weights & get sampling client
# ---------------------------
print("Saving final DPO weights and creating sampling client...")
if hasattr(dpo_client, "save_weights_and_get_sampling_client"):
    final_sampling_client = dpo_client.save_weights_and_get_sampling_client(name=SAVE_NAME_FINAL)
    print("Final sampling client created:", SAVE_NAME_FINAL)
else:
    if hasattr(dpo_client, "save_state"):
        ckpt = dpo_client.save_state()
        print("Saved state:", ckpt)
    final_sampling_client = None
    print("No save_weights_and_get_sampling_client available in SDK; check cookbook version.")

# ---------------------------
# Quick inference test if sampling client present
# ---------------------------
if final_sampling_client is not None:
    sample_prompt = "I want an acoustic, mellow song similar to early 90s rock."
    try:
        resp = final_sampling_client.sample(prompt=sample_prompt, num_samples=1, sampling_params=tinker.SamplingParams(max_tokens=64)).result()
        # try common structures
        if hasattr(resp, "samples") and len(resp.samples):
            print("Model output:\n", resp.samples[0].text)
        elif hasattr(resp, "sequences") and len(resp.sequences):
            seq = resp.sequences[0]
            txt = getattr(seq, "text", None)
            if txt:
                print("Model output:\n", txt)
            else:
                print("Model sequence tokens:", seq.tokens[:50])
        else:
            print("Sampling response (raw):", resp)
    except Exception as e:
        print("Sampling failed:", e)
else:
    print("No final sampling client to test inference.")

print("All done.")

Tinker version: 0.7.0
Loaded SFT examples: 89740


Converting SFT examples: 100%|██████████| 89740/89740 [01:06<00:00, 1352.02it/s]


Converted SFT examples -> datums: 89740
Creating SFT LoRA training client...
Training client created.

Starting SFT training...

=== SFT Epoch 1/2 ===


SFT epoch 1:   0%|          | 1/22435 [00:01<6:51:55,  1.10s/it]

  step=0/22435 | nll=4.8237 | lr=0.000100


...

SFT epoch 2:  53%|█████▎    | 12001/22435 [4:28:35<4:15:47,  1.47s/it]

  step=12000/22435 | nll=1.5652 | lr=0.000023


SFT epoch 2:  54%|█████▎    | 12021/22435 [4:29:02<3:49:26,  1.32s/it]

  step=12020/22435 | nll=1.7251 | lr=0.000023


SFT epoch 2:  54%|█████▎    | 12029/22435 [4:29:13<3:51:28,  1.33s/it]

# Evaluation

In [ ]:
# =============================================
# Evaluation Utility Functions
# =============================================
import numpy as np
import torch
from tqdm import tqdm
from typing import List, Dict

# ---------- Ranking Metrics ----------
def precision_at_k(preds: List[str], true_item: str, k: int) -> float:
    return 1.0 if true_item in preds[:k] else 0.0

def recall_at_k(preds: List[str], true_item: str, k: int) -> float:
    return 1.0 if true_item in preds[:k] else 0.0

def mrr_at_k(preds: List[str], true_item: str) -> float:
    for idx, p in enumerate(preds):
        if p == true_item:
            return 1.0 / (idx + 1)
    return 0.0

def ndcg_at_k(preds: List[str], true_item: str, k: int) -> float:
    for idx, p in enumerate(preds[:k]):
        if p == true_item:
            return 1.0 / np.log2(idx + 2)
    return 0.0


# ---------- Hallucination ----------
def hallucination_rate(preds: List[str], valid_song_ids: set) -> float:
    invalid = sum([1 for p in preds if p not in valid_song_ids])
    return invalid / len(preds)


# ---------- Pairwise DPO Accuracy ----------
def preference_accuracy(logp_chosen: float, logp_rejected: float) -> int:
    # Returns 1 if model ranks chosen higher, else 0
    return 1 if logp_chosen > logp_rejected else 0


## SFT Eval

In [ ]:
# =============================================
# SFT Evaluation: Perplexity + NLL
# =============================================

from tinker_cookbook.supervised.data import conversation_to_datum
from tinker_cookbook.supervised.common import compute_mean_nll

def evaluate_sft_ppl(sft_eval_data, renderer, tokenizer, model_client, max_length=4096):
    eval_data = []

    for ex in sft_eval_data:
        datum = conversation_to_datum(
            ex["messages"],
            renderer,
            max_length,
            train_on=renderers.TrainOnWhat.ALL_ASSISTANT_MESSAGES
        )
        eval_data.append(datum)

    logprobs_list = []
    weight_list = []

    for datum in tqdm(eval_data, desc="Evaluating SFT"):
        result = model_client.forward([datum], compute_loss=True)
        logprobs = result[0]["loss_fn_outputs"][0]["logprobs"]
        weights = datum.loss_fn_inputs["weights"]

        logprobs_list.append(logprobs)
        weight_list.append(weights)

    nll = compute_mean_nll(logprobs_list, weight_list)
    ppl = np.exp(nll)

    print(f"Mean NLL: {nll:.4f}")
    print(f"Perplexity: {ppl:.4f}")

    return nll, ppl


## DPO Eval

In [ ]:
# =============================================
# DPO Evaluation: Pairwise Preference Accuracy
# =============================================

def evaluate_dpo_accuracy(dpo_eval_data, renderer, tokenizer, model_client, max_length=4096):
    correct = 0
    total = 0

    for ex in tqdm(dpo_eval_data, desc="Evaluating DPO"):
        instruction = ex["instruction"]
        inp = ex.get("input", "")

        chosen = ex["chosen"]
        rejected = ex["rejected"]

        # Form model inputs (simple prompt format)
        prompt = instruction + "\n" + inp

        # Log-prob of chosen
        out1 = model_client.score_text(prompt, chosen)
        logp_chosen = out1["logprob"]

        # Log-prob of rejected
        out2 = model_client.score_text(prompt, rejected)
        logp_rejected = out2["logprob"]

        correct += preference_accuracy(logp_chosen, logp_rejected)
        total += 1

    acc = correct / total
    print(f"DPO Pairwise Preference Accuracy: {acc:.4f}")
    return acc


## Recommendation Ranking Metrics

In [ ]:
from collections import defaultdict

# ----------------------------------------------------
# 1. LOAD SPOTIFY FEATURE DATA WITH SEMANTIC IDs
# ----------------------------------------------------
spotify_df = df.copy()
spotify_df = spotify_df[['track_id', 'semantic_id', 'track_name', 'artists', 'track_genre']]
print("Spotify track records:", len(spotify_df))


# ----------------------------------------------------
# 2. LOAD MUSIC4ALL LISTENING HISTORY
# ----------------------------------------------------
print("Loading listening history...")
history_df = pd.read_csv(
    "listening_history.tsv.bz2",
    sep="\t",
    names=["user_id", "track_id", "timestamp"],
    compression="bz2"
)
print("History rows:", len(history_df))


# ----------------------------------------------------
# 3. LOAD MUSIC4ALL METADATA FOR TRACK MAPPING
# ----------------------------------------------------
meta_df = pd.read_csv("id_information.csv")
meta_df = meta_df[['music4all_track_id', 'spotify_track_id']]
meta_df = meta_df.dropna(subset=['spotify_track_id'])
meta_df['spotify_track_id'] = meta_df['spotify_track_id'].astype(str)
print("Metadata rows:", len(meta_df))


# ----------------------------------------------------
# 4. MERGE HISTORY WITH METADATA (MAP → SPOTIFY IDs)
# ----------------------------------------------------
history_df['track_id'] = history_df['track_id'].astype(str)

merged = history_df.merge(
    meta_df,
    left_on='track_id',
    right_on='music4all_track_id',
    how='inner'
)

merged = merged.rename(columns={'spotify_track_id': 'track_id_spotify'})
merged = merged[['user_id', 'track_id_spotify', 'timestamp']]
print("Merged listening rows:", len(merged))


# ----------------------------------------------------
# 5. FILTER TO TRACKS WE HAVE FEATURES FOR
# ----------------------------------------------------
valid_tracks = set(spotify_df['track_id'])
merged = merged[merged['track_id_spotify'].isin(valid_tracks)]
print("Filtered listening rows:", len(merged))


# ----------------------------------------------------
# 6. REPLACE track_id → semantic_id
# ----------------------------------------------------
spotify_map = dict(zip(spotify_df['track_id'], spotify_df['semantic_id']))
merged['semantic_id'] = merged['track_id_spotify'].map(spotify_map)


# ----------------------------------------------------
# 7. BUILD USER → PLAYLIST SEQUENCES
# ----------------------------------------------------
user_playlists = defaultdict(list)

for user, group in merged.groupby("user_id"):
    seq = group.sort_values("timestamp")['semantic_id'].tolist()
    seq = [s for s in seq if isinstance(s, str) and len(s) > 0]
    if len(seq) >= 5:
        user_playlists[user] = seq

print("Users with valid playlists:", len(user_playlists))


# ----------------------------------------------------
# 8. BUILD semantic_id → playlists containing it
# ----------------------------------------------------
semantic_to_playlists = defaultdict(set)

for user, playlist in user_playlists.items():
    for sid in playlist:
        semantic_to_playlists[sid].add(user)

print("Total semantic IDs with playlist context:", len(semantic_to_playlists))


# ----------------------------------------------------
# 9. BUILD song-level PLAYLIST CONTEXT
# ----------------------------------------------------
song_playlist_context = defaultdict(set)

for user, playlist in user_playlists.items():
    for sid in playlist:
        for other in playlist:
            if sid != other:
                song_playlist_context[sid].add(other)

print("Playlist context entries ready.")


# ============================================================
# 10. PLAYLIST-BASED EVALUATION METRICS
# ============================================================

def in_playlist_hit_rate(true_context, predictions):
    """Fraction of predicted songs that appear in true playlist neighbors."""
    if len(predictions) == 0:
        return 0.0
    return len(set(predictions) & set(true_context)) / len(predictions)


def in_playlist_recall(true_context, predictions):
    """Fraction of true playlist neighbors recovered by predictions."""
    if len(true_context) == 0:
        return 0.0
    return len(set(predictions) & set(true_context)) / len(true_context)


def mrr_at_k(true_next, predictions):
    """MRR for the next expected song (if you use next-song prediction)."""
    for rank, p in enumerate(predictions, start=1):
        if p == true_next:
            return 1.0 / rank
    return 0.0


def jaccard_similarity(true_context, predictions):
    inter = len(set(true_context) & set(predictions))
    union = len(set(true_context) | set(predictions))
    return inter / union if union > 0 else 0.0


# ============================================================
# 11. METRIC WRAPPER FOR A MODEL
# ============================================================

# Placeholder: plug in your LLM's inference
# It must output a *list of predicted semantic_ids*
def get_predictions(prompt, top_k=10):
    """
    Replace this function with your LLM call that maps:
        natural-language query → semantic-ID predictions
    """
    # example dummy:
    return ["<12><45><99>", "<10><22><87>", "<44><11><05>"][:top_k]


def evaluate_song(song_sid, top_k=10):
    """
    Evaluate model predictions for a given song semantic ID.
    """
    if song_sid not in song_playlist_context:
        return None  # no playlist context available

    true_context = list(song_playlist_context[song_sid])

    # Use track_name + artists if you want a real natural-language prompt
    prompt = f"Recommend songs similar to {song_sid}"

    preds = get_predictions(prompt, top_k=top_k)

    return {
        "hit_rate": in_playlist_hit_rate(true_context, preds),
        "recall": in_playlist_recall(true_context, preds),
        "jaccard": jaccard_similarity(true_context, preds),
        # "mrr": mrr_at_k(true_next_song, preds),  # if used
        "num_true_context": len(true_context),
        "num_preds": len(preds),
    }


# ============================================================
# 12. RUN EVALUATION ON SAMPLE SONGS
# ============================================================

sample_songs = list(song_playlist_context.keys())[:50]

results = []

for sid in sample_songs:
    res = evaluate_song(sid, top_k=10)
    if res:
        res["song_sid"] = sid
        results.append(res)

metrics_df = pd.DataFrame(results)
metrics_df.describe()



## Hallucination Eval

In [ ]:
# =============================================
# Hallucination Evaluation
# =============================================

def evaluate_hallucinations(test_data, model_client, valid_song_ids):
    hallucinations = []

    for ex in tqdm(test_data, desc="Hallucination Eval"):
        instruction = ex["instruction"]

        resp = model_client.generate(
            instruction,
            temperature=0.1,
            max_tokens=20
        )["text"]

        preds = resp.replace("<", "").replace(">", "").split()
        hallucinations.append(hallucination_rate(preds, valid_song_ids))

    avg_hallucination = np.mean(hallucinations)
    print(f"Hallucination Rate: {avg_hallucination:.4f}")
    return avg_hallucination
